# Catogorical Variable

- Label Encoding: work well in tree-based models when the variables are `ordinal`.
- One-Hot Encoding:
    different from `ordinal variables`, `nominal variables` do not have an intrinsic ranking

- Get list of categorical variables

In [24]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

In this exercise, you will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('input/train.csv', index_col='Id') 
X_test = pd.read_csv('input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

Use the next code cell to print the first five rows of the data.

In [26]:
X_train.head()

MSSubClass MSZoning  LotArea Street LotShape LandContour Utilities  \
Id                                                                        
619          20       RL    11694   Pave      Reg         Lvl    AllPub   
871          20       RL     6600   Pave      Reg         Lvl    AllPub   
93           30       RL    13360   Pave      IR1         HLS    AllPub   
818          20       RL    13265   Pave      IR1         Lvl    AllPub   
303          20       RL    13704   Pave      IR1         Lvl    AllPub   

    LotConfig LandSlope Neighborhood  ... OpenPorchSF EnclosedPorch 3SsnPorch  \
Id                                    ...                                       
619    Inside       Gtl      NridgHt  ...         108             0         0   
871    Inside       Gtl        NAmes  ...           0             0         0   
93     Inside       Gtl      Crawfor  ...           0            44         0   
818   CulDSac       Gtl      Mitchel  ...          59             0         0   
303    Corner       Gtl      CollgCr  ...          81             0         0   

    ScreenPorch  PoolArea  MiscVal  MoSold  YrSold SaleType SaleCondition  
Id                                                                         
619         260         0        0       7    2007      New       Partial  
871           0         0        0       8    2009       WD        Normal  
93            0         0        0       8    2009       WD        Normal  
818           0         0        0       7    2008       WD        Normal  
303           0         0        0       1    2006       WD        Normal  

[5 rows x 60 columns]

Notice that the dataset contains both numerical and categorical variables.  You'll need to encode the categorical data before training a model.

To compare different models, you'll use the same `score_dataset()` function from the tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 1: Drop columns with categorical data

You'll get started with the most straightforward approach.  Use the code cell below to preprocess the data in `X_train` and `X_valid` to remove columns with categorical data.  Set the preprocessed DataFrames to `drop_X_train` and `drop_X_valid`, respectively.  

In [ ]:
# Fill in the lines below: drop columns in training and validation data
train_categorical_columns_yn = [(cl, X_train[cl].dtypes == 'object') for cl in X_train.columns]
train_categorical_columns=[]
for cl, yn in train_categorical_columns_yn:
    if yn:
        train_categorical_columns.append(cl) 

drop_X_train = X_train.drop(train_categorical_columns,axis=1)

valid_categorical_columns_yn = [(cl, X_valid[cl].dtypes == 'object') for cl in X_valid.columns]
valid_categorical_columns=[]
for cl, yn in valid_categorical_columns_yn:
    if yn:
        valid_categorical_columns.append(cl) 

drop_X_valid = X_valid.drop(valid_categorical_columns,axis=1)

Hint: Use the select_dtypes() method to drop all columns with the object dtype.

In [70]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


# Step 2: Label encoding

Before jumping into label encoding, we'll investigate the dataset.  Specifically, we'll look at the `'Condition2'` column.  The code cell below prints the unique entries in both the training and validation sets.

In [71]:
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


If you now write code to: 
- fit a label encoder to the training data, and then 
- use it to transform both the training and validation data, 

you'll get an error.  Can you see why this is the case?  (_You'll need  to use the above output to answer this question._)

# Solution: 
- Fitting a label encoder to a column in the training data creates a corresponding integer-valued label for each unique value that appears in the training data. `In the case that the validation data contains values that don't also appear in the training data`, the encoder will throw an error, because these values won't have an integer assigned to them. Notice that the 'Condition2' column in the validation data contains the values 'RRAn' and 'RRNn', but these don't appear in the training data -- thus, if we try to use a label encoder with scikit-learn, the code will throw an error.

- This is a common problem that you'll encounter with real-world data, and there are many approaches to fixing this issue.  For instance, you can write a custom label encoder to deal with new categories.  The simplest approach, however, is to drop the problematic categorical columns.  

Run the code cell below to save the problematic columns to a Python list `bad_label_cols`.  Likewise, columns that can be safely label encoded are stored in `good_label_cols`.

In [72]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['LandSlope', 'Exterior1st', 'ExterCond', 'Condition1', 'Neighborhood', 'Utilities', 'Exterior2nd', 'Condition2', 'RoofMatl', 'HeatingQC', 'Heating', 'Functional', 'RoofStyle', 'Foundation', 'SaleType']


Use the next code cell to label encode the data in `X_train` and `X_valid`.  Set the preprocessed DataFrames to `label_X_train` and `label_X_valid`, respectively.  
- We have provided code below to drop the categorical columns in `bad_label_cols` from the dataset. 
- You should label encode the categorical columns in `good_label_cols`.  

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
